In [1]:
import numpy as np
import pandas as pd
# import dcMinMaxFunctions as dc
# import dcor
from scipy.misc import derivative
from sklearn.model_selection import train_test_split
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from scipy import stats
import wandb
from opacus import PrivacyEngine
from cov_help import *
import diffprivlib as dp

In [2]:
data_path='data/covtype.csv'
norm =1
X,Y = cov_data_loader(data_path,norm=norm)

In [3]:
 X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
batch_size = 1024
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train,Y_train), batch_size=batch_size,
                                            shuffle=True, num_workers=2,drop_last=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_test,Y_test), batch_size=batch_size,
                                            shuffle=True, num_workers=2)

In [10]:
from sklearn.datasets import make_classification
from diffprivlib.models import RandomForestClassifier
X, y = make_classification(n_samples=1000, n_features=4,
                           n_informative=2, n_redundant=0,
                           random_state=0, shuffle=False)
X = X_train.numpy()
y=  Y_train.numpy()   
clf = RandomForestClassifier(n_estimators=100,epsilon=10,max_depth=10,  random_state=0)
clf.fit(X, y)


/home/kaustubh/llm/lib/python3.10/site-packages/diffprivlib/models/forest.py:190: PrivacyLeakWarning: Bounds have not been specified and will be calculated on the data provided. This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify bounds for each dimension.
  warnings.warn("Bounds have not been specified and will be calculated on the data provided. This will "
/home/kaustubh/llm/lib/python3.10/site-packages/diffprivlib/models/forest.py:209: PrivacyLeakWarning: Classes have not been specified and will be calculated on the data provided. This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify the prediction classes for model.
  warnings.warn("Classes have not been specified and will be calculated on the data provided. This will "


RandomForestClassifier(accountant=BudgetAccountant(spent_budget=[(10, 0), (10, 0), (10, 0)]),
                       bounds=(array([ 0.19871134,  0.        ,  0.        ,  0.        , -0.01720028,
        0.        ,  0.00598271,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ]),
                               array([4.12165761e-01, 3.84602584e-02, 6.62371097e-03, 1.4465...
       1.06834050e-04, 1.06834050e-04, 1.06834050e-04, 1.06834050e-04,
       1.06834050e-04, 1.06834050e-04, 1.06834050e-04, 1.06834050e-04,
       1.06834050e-04, 1.06834050e-04, 1.06834050e-04, 1.06834050e-04,
       1.06834050e-04, 1.06834050e-04, 1.06834050e-04, 1.06834050e-04,
       1.06834050e-04, 1.06834050e-04])),
                       classes=array([0, 1, 2, 3, 4, 5, 6]), epsilon=10,
                       max_depth=10, n_estimators=100, random_state=0)

In [7]:
(clf.predict(X)==y).sum()/len(y)

0.871

In [46]:
# from sklearn.datasets import make_classification
# from diffprivlib.models import RandomForestClassifier
# # X, y = make_classification(n_samples=10000, n_features=4,
# #                            n_informative=2, n_redundant=0,
# #                            random_state=0, shuffle=False)
clf = RandomForestClassifier(n_estimators=100,epsilon= 100, random_state=0)
clf.fit(X, y)
clf.predict(X_test.numpy()[0:20])

/home/kaustubh/llm/lib/python3.10/site-packages/diffprivlib/models/forest.py:190: PrivacyLeakWarning: Bounds have not been specified and will be calculated on the data provided. This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify bounds for each dimension.
  warnings.warn("Bounds have not been specified and will be calculated on the data provided. This will "
/home/kaustubh/llm/lib/python3.10/site-packages/diffprivlib/models/forest.py:209: PrivacyLeakWarning: Classes have not been specified and will be calculated on the data provided. This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify the prediction classes for model.
  warnings.warn("Classes have not been specified and will be calculated on the data provided. This will "


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [36]:
X

array([[-0.85255585,  1.00312536,  0.53286612,  0.76440094],
       [-0.16171265,  1.55666395, -0.85608336, -1.6823405 ],
       [-1.36883328,  0.11776563,  0.65306946,  0.93025508],
       ...,
       [ 1.46718799,  1.63720032,  1.23934523,  0.3642774 ],
       [ 1.07113253,  1.15113659, -1.16663743, -0.50382808],
       [ 1.63564375,  2.24896687,  0.11141039, -1.53470286]])

In [45]:
X = X_train.numpy()
y=  Y_train.numpy() 

In [39]:
X

array([[3.2370716e-01, 3.8139757e-02, 1.2820086e-03, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [3.4934735e-01, 3.4293730e-02, 1.6025108e-03, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [3.5693255e-01, 3.5575740e-02, 1.0683405e-03, ..., 1.0683405e-04,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [3.4133479e-01, 8.2262224e-03, 9.6150645e-04, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [3.2723269e-01, 1.2285916e-02, 2.0298469e-03, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [2.9966950e-01, 3.9528599e-03, 2.2435151e-03, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]], dtype=float32)

0.7675394247111723

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
# X, y = make_classification(n_samples=1000, n_features=4,
#                            n_informative=2, n_redundant=0,
#                            random_state=0, shuffle=False)
clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0)
clf.fit(X, y)

RandomForestClassifier(max_depth=10, random_state=0)

In [9]:
(clf.predict(X) == y).sum()/len(y)

1.0

In [4]:
criterion = nn.CrossEntropyLoss()


model = nn.Sequential(
        nn.Linear(54, 64),
        nn.ReLU(),
        nn.Linear(64, 128),
        nn.ReLU(),
        nn.Linear(128, 256),
        nn.ReLU(),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Linear(128, 7),
        nn.Softmax(dim=1)

    )
learning_rate =0.001
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate,weight_decay=1e-4)


# enter PrivacyEngine
privacy_engine = PrivacyEngine()
model2, optimizer2, data_loader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    target_epsilon=eps,
    target_delta =0.0001,
    epochs = num_epochs,
    max_grad_norm=1.0,
)
# train_emb(model2, data_loader, criterion, optimizer2, num_epochs=num_epochs,device=torch.device('cuda'),test_loader = test_loader)

/home/kaustubh/llm/lib/python3.10/site-packages/opacus/privacy_engine.py:142: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(


NameError: name 'train_loader' is not defined

In [2]:
run = wandb.init(project='cov_type_priv_base')

11/17/2023 18:23:31:ERROR:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ponkshekaustubh11. Use `wandb login --relogin` to force relogin
